# Tutorial: shRNA guide analysis

In this small case study we will do the following:
    
- create a subset of the human transcriptome and load the gene sequences
- create a random list of shRNA guide sequences (in a real scenario those would, e.g., predicted by some external tool)
- create a pandas dataframe containing 
    - the guide sequence 
    - a list of transcript ids that contain this guide (exact match) in their spliced RNA sequence
    - a list of gene ids for these transcripts
- filter guides that are not found or that target multiple genes
- check for untargeted transcripts of the targeted genes

Please note that the code in this case study is not optimized and is more explicit than necessary in order to showcase the API and make the example easier to understand. 

*Required resources (see above for instructions how to download):*
- Human genome FASTA (GRCh38), accessible at https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000001405.26/
- Full gencode annotation gff3 file (sorted), available at https://www.gencodegenes.org/human/

In [9]:
# set path and load pygenlib
import os, pathlib, platform
PYGENLIB_SRC=pathlib.Path('/Users/niko/projects/pygenlib/') 
os.chdir(PYGENLIB_SRC)
# install libraries. Recommended to run in a venv here!
#!{sys.executable} -m pip install -r requirements.txt 
display(f"Running pygenlib on python {platform.python_version()}. Using pygenlib code from {PYGENLIB_SRC}")
# load pygenlib
import pygenlib as pg
from pygenlib import SEP, display_textarea
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import traceback
import math
import random

'Running pygenlib on python 3.10.4. Using pygenlib code from /Users/niko/projects/pygenlib'

First, we download the required resources.
NOTE that this needs bedtools, samtools and htslib (bgzip, tabix) installed.
Total size of the downloaded data (for all tutorials) is ~150M. Files are only downloaded if not existing already in the `notebooks/large_test_resources/` directory.

In [2]:
import traceback
from pygenlib.testdata import download_bgzip_slice
outdir=PYGENLIB_SRC / 'notebooks/large_test_resources' # update to your preferred location
large_test_resources = {
    "outdir": f"{outdir}", # update to your preferred location
    "resources": {
        # -------------- Full gencode39 annotation -------------------------------
        "full_gencode_gff": {
            "uri": "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/gencode.v39.annotation.gff3.gz",
            "filename": "gencode_39.gff3.gz",
            "recreate": False
        },
        # -------------- GRCh38 chr20 -------------------------------
        "grch38_chr20": {
            "uri": "https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr20.fa.gz",
            "filename": "grch38_chr20.fa.gz",
            "recreate": False
        }
    }
}
display(f'Downloading test data files to {outdir}')
for resname in large_test_resources['resources']:
    try:
        download_bgzip_slice(large_test_resources, resname, view_tempdir=False)
    except Exception:
        display(traceback.format_exc())
        display(f"Error creating resource {resname}. Some tests may not work...")
display("All done.")

'Downloading test data files to /Users/niko/projects/pygenlib/notebooks/large_test_resources'

Creating testdataset full_gencode_gff
Resource already exists, skipping...
Creating testdataset grch38_chr20
Resource already exists, skipping...


'All done.'

In [7]:
# please update paths accordingly
grch38_fasta=pg.get_resource("grch38_chr20", conf=large_test_resources) # see download section above
gencode_gff=pg.get_resource("full_gencode_gff", conf=large_test_resources) # see download section above

# Build subset of human transcriptome (chr20)
config={
    'genome_fa': grch38_fasta,
    'annotation_gff': gencode_gff,
    'annotation_flavour': 'gencode',
    'copied_fields': ['gene_type'],
    'load_sequences': True
}
t=pg.Transcriptome(config, pg.TranscriptFilter().include_chromosomes(['chr20']))
display(t)

Building transcriptome (1 chromosomes)
:   0%|          | 0/1 [00:00<?, ?it/s]

Load sequences:   0%|          | 0/1480 [00:00<?, ?it/s]

Build interval trees:   0%|          | 0/1480 [00:00<?, ?it/s]

Transcriptome with 1480 genes and 5822 tx (+seq)

In [11]:
# create a random set of shRNA guides of length 10. In a real scenario those would, e.g., be predicted by some external tool.
# Here, we use rnd_seq, a convenience pygenlib method, to create 3 random guide sequences with low GC%. 
random.seed(0) # if you change this, different random sequences will be created
guides=pg.rnd_seq(10, 'GC'* 30 + 'AT' * 70, 20) 
print(guides)

['AATGAAATTA', 'TTCTCACTGA', 'AAAACTCAGT', 'AATCATAATG', 'TATTACACCA', 'ATTAGAATAA', 'TAGGTGTGTA', 'CGTATCTTAA', 'CATAGAAATT', 'CAATTTACGC', 'GAAATCGTTC', 'GCGTAAAAAA', 'TACTATAATA', 'TATTCTACAA', 'TTGAGTTCAG', 'TCTTCCCAGC', 'AAAAAGGCAG', 'TTGACACCTC', 'CCTCACACTC', 'TAACACGGTT']


In [12]:
# now we search for transcripts that contain respective kmers in their spliced RNA seq. 
# To make this fast, we first search for the kmer in the respective gene sequence (candidate_genes) and for those
# check all spliced tx sequences (overlapping_tx).
# Finally, we are interested whether some guides bind RNAs from multiple genes and create a set of gene ids for the overlapping genes (overlapping_genes).
# We combine the results in a pandas dataframe
d=[]
for guide in tqdm(guides, total=len(guides), desc='analyzing guide'):
    candidate_genes={g for g in t.genes if guide in g.sequence}
    overlapping_tx={tx for g in candidate_genes for tx in g.transcript if guide in tx.spliced_sequence}
    overlapping_genes={tx.parent.feature_id for tx in overlapping_tx}
    d.append(
        {
            'guide_seq': guide,
            'gids': ','.join(overlapping_genes),
            'n_gids': len(overlapping_genes),
            'tids': ','.join([tx.feature_id for tx in overlapping_tx])
        }
    )
df=pd.DataFrame(d)
display(df.head(8))

analyzing guide:   0%|          | 0/20 [00:00<?, ?it/s]

,guide_seq,gids,n_gids,tids
0,AATGAAATTA,"ENSG00000227906.9,ENSG00000125971.17,ENSG00000...",7,"ENST00000688800.1,ENST00000666905.1,ENST000006..."
1,TTCTCACTGA,"ENSG00000228340.7,ENSG00000101298.15",2,"ENST00000614659.1,ENST00000381867.6,ENST000004..."
2,AAAACTCAGT,"ENSG00000089101.19,ENSG00000101109.12,ENSG0000...",3,"ENST00000499879.6,ENST00000674269.1,ENST000003..."
3,AATCATAATG,ENSG00000054796.13,1,"ENST00000345868.8,ENST00000371260.8,ENST000004..."
4,TATTACACCA,ENSG00000088930.8,1,ENST00000377191.5
5,ATTAGAATAA,ENSG00000101126.18,1,ENST00000371602.9
6,TAGGTGTGTA,ENSG00000228293.1,1,ENST00000418739.1
7,CGTATCTTAA,,0,


In [13]:
# In the DF above, we can see that some guides (e.g., CGTATCTTAA) are not found (no gids) while some target multiple 
# genes (e.g., AATGAAATTA). This is expected here as we generate short (10bp) sequences with low GC%/sequence complexity
# that are likely found in many genomic locations. In a real scenario, one would expect few such cases for guides predicted by SOTA tools.
# We continue the analysis by filtering those bad guides...
fil=df[df['n_gids']==1].copy() 
display(fil)

,guide_seq,gids,n_gids,tids
3,AATCATAATG,ENSG00000054796.13,1,"ENST00000345868.8,ENST00000371260.8,ENST000004..."
4,TATTACACCA,ENSG00000088930.8,1,ENST00000377191.5
5,ATTAGAATAA,ENSG00000101126.18,1,ENST00000371602.9
6,TAGGTGTGTA,ENSG00000228293.1,1,ENST00000418739.1
10,GAAATCGTTC,ENSG00000149609.6,1,ENST00000375222.4
12,TACTATAATA,ENSG00000088812.18,1,ENST00000262919.10
13,TATTCTACAA,ENSG00000231290.6,1,"ENST00000448374.5,ENST00000447767.1"
14,TTGAGTTCAG,ENSG00000172264.18,1,ENST00000490428.5


In [15]:
# ...and now we want to check for the remaining ones whether all tx of the respective genes are targeted.
# For this, we query the transcriptome for sets of transcript ids annotated for a given gene and use the 
# pygenlib cmp_sets() method to get shared and unique items when comparing to the set of transcript ids we 
# found to be targeted by each guide.
# Finally, we add the number of missed (untargeted) transcripts and the respective gene names to the dataframe.
missed_tx, gene_names=[],[]
for guide, gid, tids in zip(fil['guide_seq'], fil['gids'], fil['tids']):
    all_tid={tx.feature_id for tx in t.gene[gid].transcript}
    found_tids=set(tids.split(','))
    # cmp_sets is a pygenlib utility function for set comparison
    s,m,w=pg.cmp_sets(all_tid, found_tids)
    missed_tx.append(f"{len(m)}/{len(all_tid)}")
    gene_names.append(t.gene[gid].gene_name)
    assert len(w)==0, "We should not find a tx that was not found before"
fil['missed_tx']=missed_tx
fil['gene_name']=gene_names
display(fil)

,guide_seq,gids,n_gids,tids,missed_tx,gene_name
3,AATCATAATG,ENSG00000054796.13,1,"ENST00000345868.8,ENST00000371260.8,ENST000004...",1/5,SPO11
4,TATTACACCA,ENSG00000088930.8,1,ENST00000377191.5,0/1,XRN2
5,ATTAGAATAA,ENSG00000101126.18,1,ENST00000371602.9,8/9,ADNP
6,TAGGTGTGTA,ENSG00000228293.1,1,ENST00000418739.1,0/1,ENSG00000228293
10,GAAATCGTTC,ENSG00000149609.6,1,ENST00000375222.4,1/2,C20orf144
12,TACTATAATA,ENSG00000088812.18,1,ENST00000262919.10,1/2,ATRN
13,TATTCTACAA,ENSG00000231290.6,1,"ENST00000448374.5,ENST00000447767.1",7/9,APCDD1L-DT
14,TTGAGTTCAG,ENSG00000172264.18,1,ENST00000490428.5,15/16,MACROD2


We can see that we are not targetting all annotated transcripts except for XRN2 (which has only 1 tx).
In a real scenario, we would possssibly do such a check only on tx that are actually expressed in the respective
cells. 